In [13]:
import cx_Oracle
import pandas as pd

In [14]:
credentials = os.environ.get('CONN_ORACLETTCS')
db = cx_Oracle.connect(credentials)
cursor = db.cursor()
fd = open('new_biz.sql', 'r')
sql = fd.read()
fd.close()
df = pd.read_sql_query(sql,db)
df.to_csv('/usr/local/airflow/poseidon/data/temp/ttcs/ttcs_last_q.csv',index=False,encoding='utf-8')
fd = open('active_biz.sql', 'r')
sql = fd.read()
fd.close()
df = pd.read_sql_query(sql,db)
df.to_csv('/usr/local/airflow/poseidon/data/temp/ttcs/active.csv',index=False,encoding='utf-8')

In [15]:
last_quarter = pd.read_csv('/usr/local/airflow/poseidon/data/temp/ttcs/ttcs_last_q.csv')
active = pd.read_csv('/usr/local/airflow/poseidon/data/temp/ttcs/active.csv')
dfs = [last_quarter, active]
names = ['last_quarter', 'active']
for df, name in zip(dfs, names):
    df.columns = [x.lower() for x in df.columns]
    df = df.apply(lambda x: x.astype(str).str.lower())
    df['account_key'] = df['account_key'].astype(int)
    df['naics_code'] = df['naics_code'].astype(int)
    df['expiration_dt'] = pd.to_datetime(df['expiration_dt'],errors='coerce').dt.date
    df['creation_dt'] = pd.to_datetime(df['creation_dt'],errors='coerce').dt.date
    df['bus_start_dt'] = pd.to_datetime(df['bus_start_dt'],errors='coerce').dt.date
    df['state'] = df['state'].str.upper()
    df = df.drop('num_employees',1) 
    export = '/usr/local/airflow/poseidon/data/prod/ttcs/{0}.csv'.format(name)
    df.to_csv(export,index=False,encoding='utf-8')
    